In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import datetime
import os

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

import greenhouse

In [3]:
#initializing firestore
cert_path = '/Users/dzianis/github/' + 'findremote-firebase-adminsdk-p9cw7-633a39d4a9.json'
cred = credentials.Certificate(cert_path)
firebase_admin.initialize_app(cred)

# connect to db
db = firestore.client()




# get data from FireStore
listings_db = db.collection(u'listings').stream()

listings = dict()
for listing in listings_db:
    listings[listing.id] = listing.to_dict()

In [6]:
companies = {
    'github': 'GitHub', 
    'gitlab': 'GitLab', 
    'invision': 'InVision', 
    'blockchain': 'Blockchain', 
    'automatticcareers': 'Automattic', 
    'monzo': 'Monzo', 
    'mozilla': 'Mozilla', 
    'autoscout24': 'Autoscout24',
    'zapiercareers': 'Zapier' 
}



new_listings = []

url_base = 'https://boards.greenhouse.io'
for company in companies.keys():
    company_url = url_base + '/' + company
    response = requests.get(company_url)
    time.sleep(5)
    
    response_html = BeautifulSoup(response.content, 'html.parser')
    
    #company name
    company_name = companies[company]
    
    # openings
    results = response_html.find_all('div', class_='opening')
    for result in results:
        job_id = greenhouse.get_job_id(result)
        new_listings.append(job_id)
        

print('lenght of new listings: ', len(new_listings))
print('lenght of listings: ', len(listings.keys()))

lenght of new listings:  694
lenght of listings:  741


In [25]:
# if past listings not in new listings set status "archived" 
for listing in listings.keys():
    if listing not in new_listings:
        job_data = db.collection(u'listings').document(listing).get().to_dict()
        job_data['status'] = 'archived'
        # write to Fire Store (Content)
        doc_ref = db.collection(u'listings').document(job_data['job_id'])
        doc_ref.set(job_data) 

{'job_id': '3378029',
 'location_simp': 'Remote',
 'status': 'archived',
 'ds': False,
 'em': False,
 'tags_short': ['wordpress', 'git'],
 'pm': False,
 'job_description': '<div class="" id="content">\n<p><a href="https://wpvip.com/"><span style="font-weight: 400;">WordPress VIP</span></a><span style="font-weight: 400;"> and </span><a href="https://www.parse.ly/"><span style="font-weight: 400;">Parse.ly</span></a><span style="font-weight: 400;"> are part of the </span><a href="https://automattic.com/"><span style="font-weight: 400;">Automattic</span></a><span style="font-weight: 400;"> family of brands. Together, we bring the ease and agility of WordPress to the world’s largest companies. We get to work with customers like Facebook, Salesforce, and Disney and help them create beautiful digital experiences. We are scaling fast and this is an opportunity to join us at this exciting stage of company growth!</span><span style="font-weight: 400;"><br/></span><span style="font-weight: 400;">

In [22]:
old_listings = []
for lising in listings.keys():
    
    old_listings.append(listings[lising]['job_id'])

In [24]:
for old_listing in old_listings:
    if old_listing not in new_listings:
        print(old_listing)

3378029
3578148
3596985
3726506
3903414
3906560
3906561
3980452
4023478
4023479
4026371
4048890
4048891
4098501
4145308
4173399
4198822
4216392
4266047
4282193
4339744
4409707
4409708
4430074
4439879
4442600
4449274
4454909
4460464
5175522003
5177192003
5220287003
5591853002
5836422002
5846677002
5939088002
5939092002
5939098002
5940081002
5968406002
6093527002
6180965002
6197086002
6199259002
6233617002
6236567002
6246819002
6260653002
6260665002
6260969002
6266188002
6297420002
6298867002
6305213002


In [10]:
listing.to_dict()

{'company_name': 'GitLab',
 'swe': False,
 'location': 'Remote, Americas',
 'job_url': 'https://boards.greenhouse.io/gitlab/jobs/6322712002',
 'em': False,
 'ds': False,
 'job_id': '6322712002',
 'pm': False,
 'location_simp': 'Remote, Americas',
 'mr': False,
 'dis': False,
 'job_name': 'Senior Benefits Program Manager',
 'tags_short': ['git', 'performance', 'express', 'math'],
 'de': False,
 'img_url': 'https://storage.googleapis.com/findremote/gitlab.jpg',
 'datetime': DatetimeWithNanoseconds(2022, 8, 16, 19, 24, 47, 661552, tzinfo=<UTC>),
 'tags': ['git', 'performance', 'express', 'math'],
 'status': 'active',
 'job_description': '<div class="" id="content">\n<div class="content-intro"><p>The GitLab <a class="editor-rtfLink" href="https://about.gitlab.com/solutions/devops-platform/" target="_blank">DevOps platform</a>\xa0empowers 100,000+ organizations to deliver software faster and more efficiently. We are one of the\xa0<a class="editor-rtfLink" href="https://about.gitlab.com/comp